# Introduction

The goal of text classification is to automatically classify the text documents into one or more defined categories. Some examples of text classification are:
- Understanding audience sentiment from social media,
- Detection of spam and non-spam emails,
- Auto tagging of customer queries, and
- Categorization of news articles into defined topics. <br> <br>

Text Classification is an example of supervised machine learning task since a labelled dataset containing text documents and their labels is used for train a classifier. There are 4 steps that we need to do as follows:
- Dataset Preparation (Preprocessing Data)
- Feature Engineering (Preprocessing Data)
- Model Training
- Improve Performance 


In this tutorial, we will implement Text Classifier Model for newpapers in Vietnamese. <br>
There are totally 10 classes in data set.

# Preprocessing Data

Dataset was downloaded from https://github.com/duyvuleo/VNTC

In [2]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
from keras.layers import *

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Dataset preparation

In [3]:
from pyvi import ViTokenizer, ViPosTagger
from tqdm import tqdm
import numpy as np
import gensim
import numpy as np

In [ ]:
import os 
dir_path = os.path.dirname(os.path.realpath(os.getcwd()))
dir_path = os.path.join(dir_path, 'Data')
# '/Users/macos/Desktop/Github/NLP/Text Classifier'
# Load data from dataset folder
# VNTC-master/Data/10Topics/Ver1.1/Train_Full
# VNTC-master/Data/10Topics/Ver1.1/Test_Full
def get_data(folder_path):
    X = []
    y = []
    dirs = os.listdir(folder_path)
    for path in dirs:
        file_paths = os.listdir(os.path.join(folder_path, path))
        for file_path in tqdm(file_paths):
            with open(os.path.join(folder_path, path, file_path), 'r', encoding="utf-16") as f:
                lines = f.readlines()
                lines = ' '.join(lines)
                lines = gensim.utils.simple_preprocess(lines)
                lines = ' '.join(lines)
                lines = ViTokenizer.tokenize(lines)
#                 sentence = ' '.join(words)
#                 print(lines)
                X.append(lines)
                y.append(path)
#             break
#         break
    return X, y

train_path = os.path.join(dir_path, 'VNTC-master/Data/10Topics/Ver1.1/Train_Full')
X_data, y_data = get_data(train_path)


In [ ]:
import pickle

pickle.dump(X_data, open('data/X_data.pkl', 'wb'))
pickle.dump(y_data, open('data/y_data.pkl', 'wb'))

In [ ]:
test_path = os.path.join(dir_path, 'VNTC-master/Data/10Topics/Ver1.1/Test_Full')
X_test, y_test = get_data(test_path)


  4%|▎         | 231/6250 [00:22<09:34, 10.48it/s]

In [ ]:
pickle.dump(X_test, open('data/X_test.pkl', 'wb'))
pickle.dump(y_test, open('data/y_test.pkl', 'wb'))

## Feature Engineering

In this step, raw text data will be transformed into eature vectors and new features will be created using the existing dataset. We will implement some idea as follows:
1. Count Vectors as features
2. TF-IDF Vectors as features<br>
    2.1. Word level<br>
    2.2. N-Gram level<br>
    2.3. Character level
3. Word Embeddings as features
4. Text / NLP based features
5. Topic Models as features

In [4]:
import pickle

X_data = pickle.load(open('data/X_data.pkl', 'rb'))
y_data = pickle.load(open('data/y_data.pkl', 'rb'))

X_test = pickle.load(open('data/X_test.pkl', 'rb'))
y_test = pickle.load(open('data/y_test.pkl', 'rb'))

FileNotFoundError: [Errno 2] No such file or directory: 'data/X_test.pkl'

### Count Vectors as features
Count Vector is a matrix notation of the dataset in which every row represents a document from the corpus, every column represents a term from the corpus, and every cell represents the frequency count of a particular term in a particular document.

In [ ]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(X_data)

# transform the training and validation data using count vectorizer object
X_data_count = count_vect.transform(X_data)
X_test_count = count_vect.transform(X_test)

### TF-IDF Vectors

TF(t) = (Number of times term t appears in a document) / (Total number of terms in the document)<br>
IDF(t) = log_e(Total number of documents / Number of documents with term t in it)<br>
TF-IDF Vectors can be generated at different levels of input tokens (words, characters, n-grams)

a. Word Level TF-IDF : Matrix representing tf-idf scores of every term in different documents

b. N-gram Level TF-IDF : N-grams are the combination of N terms together. This Matrix representing tf-idf scores of N-grams

c. Character Level TF-IDF : Matrix representing tf-idf scores of character level n-grams in the corpus



In [12]:
# word level - we choose max number of words equal to 30000 except all words (100k+ words)
tfidf_vect = TfidfVectorizer(analyzer='word', max_features=30000)
tfidf_vect.fit(X_data) # learn vocabulary and idf from training set
X_data_tfidf =  tfidf_vect.transform(X_data)
# assume that we don't have test set before
X_test_tfidf =  tfidf_vect.transform(X_test)

In [ ]:
tfidf_vect.get_feature_names()

In [41]:
# ngram level - we choose max number of words equal to 30000 except all words (100k+ words)
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', max_features=30000, ngram_range=(2, 3))
tfidf_vect_ngram.fit(X_data)
X_data_tfidf_ngram =  tfidf_vect_ngram.transform(X_data)
# assume that we don't have test set before
X_test_tfidf_ngram =  tfidf_vect_ngram.transform(X_test)

In [ ]:
tfidf_vect_ngram.get_feature_names()

In [4]:
# ngram-char level - we choose max number of words equal to 30000 except all words (100k+ words)
tfidf_vect_ngram_char = TfidfVectorizer(analyzer='char', max_features=30000, ngram_range=(2, 3))
tfidf_vect_ngram_char.fit(X_data)
X_data_tfidf_ngram_char =  tfidf_vect_ngram_char.transform(X_data)
# assume that we don't have test set before
X_test_tfidf_ngram_char =  tfidf_vect_ngram_char.transform(X_test)

#### Transform by SVD to decrease number of dimensions

##### Word Level

In [19]:
from sklearn.decomposition import TruncatedSVD

In [20]:
svd = TruncatedSVD(n_components=300, random_state=42)
svd.fit(X_data_tfidf)

TruncatedSVD(algorithm='randomized', n_components=300, n_iter=5,
       random_state=42, tol=0.0)

In [21]:
X_data_tfidf_svd = svd.transform(X_data_tfidf)
X_test_tfidf_svd = svd.transform(X_test_tfidf)

##### ngram Level

In [42]:
svd_ngram = TruncatedSVD(n_components=300, random_state=42)
svd_ngram.fit(X_data_tfidf_ngram)

TruncatedSVD(algorithm='randomized', n_components=300, n_iter=5,
       random_state=42, tol=0.0)

In [43]:
X_data_tfidf_ngram_svd = svd_ngram.transform(X_data_tfidf_ngram)
X_test_tfidf_ngram_svd = svd_ngram.transform(X_test_tfidf_ngram)

##### ngram Char Level

In [7]:
svd_ngram_char = TruncatedSVD(n_components=300, random_state=42)
svd_ngram_char.fit(X_data_tfidf_ngram_char)

TruncatedSVD(algorithm='randomized', n_components=300, n_iter=5,
       random_state=42, tol=0.0)

In [9]:
X_data_tfidf_ngram_char_svd = svd_ngram_char.transform(X_data_tfidf_ngram_char)
X_test_tfidf_ngram_char_svd = svd_ngram_char.transform(X_test_tfidf_ngram_char)

### Word Embeddings

We will convert each word in document to a embedding vector. We will use pretrained model for Vietnamese. The model can be downloaded from https://github.com/Kyubyong/wordvectors

Assume that, one document have $n$ word, each word is represented by 300 dimensional vector, then the document vector be 2-dimensional matrix with size $ n \times 300 $. From that, we can use DNN, RNN, CNN model for this type of data.

In [14]:
from gensim.models import KeyedVectors 
dir_path = os.path.dirname(os.path.realpath(os.getcwd()))
word2vec_model_path = os.path.join(dir_path, "Data/vi/vi.vec")

w2v = KeyedVectors.load_word2vec_format(word2vec_model_path)
vocab = w2v.wv.vocab
wv = w2v.wv

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys


In [45]:
def get_word2vec_data(X):
    word2vec_data = []
    for x in X:
        sentence = []
        for word in x.split(" "):
            if word in vocab:
#                 print(word)
                sentence.append(wv[word])

        word2vec_data.append(sentence)
#         break
    return word2vec_data

X_data_w2v = get_word2vec_data(X_data)
X_test_w2v = get_word2vec_data(X_test)



### Text / NLP based features
Idea from https://www.analyticsvidhya.com/blog/2018/04/a-comprehensive-guide-to-understand-and-implement-text-classification-in-python/

A number of extra text based features can also be created which sometimes are helpful for improving text classification models. Some examples are:

1. Word Count of the documents – total number of words in the documents
2. Character Count of the documents – total number of characters in the documents
3. Average Word Density of the documents – average length of the words used in the documents
4. Puncutation Count in the Complete Essay – total number of punctuation marks in the documents
5. Upper Case Count in the Complete Essay – total number of upper count words in the documents
6. Title Word Count in the Complete Essay – total number of proper case (title) words in the documents
7. Frequency distribution of Part of Speech Tags:
    - Noun Count
    - Verb Count
    - Adjective Count
    - Adverb Count
    - Pronoun Count
    
These features are highly experimental ones and should be used according to the problem statement only.

### Topic Models as features

Topic Modelling is a technique to identify the groups of words (called a topic) from a collection of documents that contains best information in the collection. I have used Latent Dirichlet Allocation for generating Topic Modelling Features. LDA is an iterative model which starts from a fixed number of topics. Each topic is represented as a distribution over words, and each document is then represented as a distribution over topics. Although the tokens themselves are meaningless, the probability distributions over words provided by the topics provide a sense of the different ideas contained in the documents

### Convert y to categorical

In [21]:
encoder = preprocessing.LabelEncoder()
y_data_n = encoder.fit_transform(y_data)
y_test_n = encoder.fit_transform(y_test)

In [22]:
encoder.classes_

array(['Chinh tri Xa hoi', 'Doi song', 'Khoa hoc', 'Kinh doanh',
       'Phap luat', 'Suc khoe', 'The gioi', 'The thao', 'Van hoa',
       'Vi tinh'], dtype='<U16')

# Model

In this tutorial, we will implement some models and compare them to find the most effective model for text classification problem. We will implement these models:
1. Naive Bayes Classifier
2. Linear Classifier
3. Support Vector Machine
4. Bagging Models
5. Boosting Models
6. Shallow Neural Networks
7. Deep Neural Networks
    - Convolutional Neural Network (CNN)
    - Long Short Term Modelr (LSTM)
    - Gated Recurrent Unit (GRU)
    - Bidirectional RNN
    - Recurrent Convolutional Neural Network (RCNN)
    - Other Variants of Deep Neural Networks
8. Doc2Vec model

We use the prototype function to do some classifiers as follows: <br>
(Because of my machine memory, I test only on WORD LEVEL TF-IDF (with SVD or not))

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
def train_model(classifier, X_data, y_data, X_test, y_test, is_neuralnet=False, n_epochs=3):       
    X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size=0.1, random_state=42)
    
    if is_neuralnet:
        classifier.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=n_epochs, batch_size=512)
        
        val_predictions = classifier.predict(X_val)
        test_predictions = classifier.predict(X_test)
        val_predictions = val_predictions.argmax(axis=-1)
        test_predictions = test_predictions.argmax(axis=-1)
    else:
        classifier.fit(X_train, y_train)
    
        train_predictions = classifier.predict(X_train)
        val_predictions = classifier.predict(X_val)
        test_predictions = classifier.predict(X_test)
        
    print("Validation accuracy: ", metrics.accuracy_score(val_predictions, y_val))
    print("Test accuracy: ", metrics.accuracy_score(test_predictions, y_test))

## Naive Bayes

In [13]:
train_model(naive_bayes.MultinomialNB(), X_data_tfidf, y_data, X_test_tfidf, y_test, is_neuralnet=False)

Train accuracy:  0.880031596616529
Validation accuracy:  0.8690758293838863
Test accuracy:  0.8650666031405714


In [ ]:
train_model(naive_bayes.MultinomialNB(), X_data_tfidf_ngram_svd, y_data, X_test_tfidf_ngram_svd, y_test, is_neuralnet=False)

In [ ]:
train_model(naive_bayes.MultinomialNB(), X_data_tfidf_ngram_char_svd, y_data, X_test_tfidf_ngram_char_svd, y_test, is_neuralnet=False)

### Other type Naive Bayes

In [58]:
# use too much memory
# train_model(naive_bayes.GaussianNB(), X_data_tfidf.todense(), y_data, X_test_tfidf.todense(), y_test, is_neuralnet=False)

In [15]:
train_model(naive_bayes.BernoulliNB(), X_data_tfidf, y_data, X_test_tfidf, y_test, is_neuralnet=False)

Train accuracy:  0.8485995457986374
Validation accuracy:  0.8293838862559242
Test accuracy:  0.8531554602664125


In [16]:
train_model(naive_bayes.BernoulliNB(), X_data_tfidf_svd, y_data, X_test_tfidf_svd, y_test, is_neuralnet=False)

Train accuracy:  0.8087746437152354
Validation accuracy:  0.8033175355450237
Test accuracy:  0.8143449864014453


## Linear Classifier

In [17]:
train_model(linear_model.LogisticRegression(), X_data_tfidf, y_data, X_test_tfidf, y_test, is_neuralnet=False)

Train accuracy:  0.9473060593094823
Validation accuracy:  0.9167654028436019
Test accuracy:  0.9207511960772636


In [18]:
train_model(linear_model.LogisticRegression(), X_data_tfidf_svd, y_data, X_test_tfidf_svd, y_test, is_neuralnet=False)

Train accuracy:  0.9023467070401211
Validation accuracy:  0.8927725118483413
Test accuracy:  0.9046314493875688


## SVM Model

In [11]:
train_model(svm.SVC(), X_data_tfidf_svd, y_data, X_test_tfidf_svd, y_test, is_neuralnet=False)

Train accuracy:  0.43359773557581544
Validation accuracy:  0.4277251184834123
Test accuracy:  0.3908840053203105


## Bagging Model

In [12]:
train_model(ensemble.RandomForestClassifier(), X_data_tfidf_svd, y_data, X_test_tfidf_svd, y_test, is_neuralnet=False)

Train accuracy:  0.9962479017871836
Validation accuracy:  0.8311611374407583
Test accuracy:  0.834435114049193


## Boosting Model

In [13]:
train_model(xgboost.XGBClassifier(), X_data_tfidf_svd, y_data, X_test_tfidf_svd, y_test, is_neuralnet=False)

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Train accuracy:  0.9042885824309647
Validation accuracy:  0.8696682464454977
Test accuracy:  0.8786850098266928


/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


## Deep Neural Network

In [112]:
from keras.layers import *

In [24]:
def create_dnn_model():
    input_layer = Input(shape=(300,))
    layer = Dense(1024, activation='relu')(input_layer)
    layer = Dense(1024, activation='relu')(layer)
    layer = Dense(512, activation='relu')(layer)
    output_layer = Dense(10, activation='softmax')(layer)
    
    classifier = models.Model(input_layer, output_layer)
    classifier.compile(optimizer=optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return classifier

In [48]:
classifier = create_dnn_model()
train_model(classifier=classifier, X_data=X_data_tfidf_svd, y_data=y_data_n, X_test=X_test_tfidf_svd, y_test=y_test_n, is_neuralnet=True)

Train on 30383 samples, validate on 3376 samples
Epoch 1/3
30383/30383 [==============================] - 33s 1ms/step - loss: 0.4122 - acc: 0.8666 - val_loss: 0.2976 - val_acc: 0.8960
Epoch 2/3
30383/30383 [==============================] - 33s 1ms/step - loss: 0.2560 - acc: 0.9149 - val_loss: 0.2956 - val_acc: 0.9023
Epoch 3/3
30383/30383 [==============================] - 31s 1ms/step - loss: 0.1987 - acc: 0.9322 - val_loss: 0.3094 - val_acc: 0.8969
Validation accuracy:  0.8969194312796208
Test accuracy:  0.9038770770055387


## Convolutional Neural Network 

In [49]:
def create_cnn_model():
    pass

## Recurrent Neural Network  

### LSTM 

In [68]:
def create_lstm_model():
    input_layer = Input(shape=(300,))
    
    layer = Reshape((10, 30))(input_layer)
    layer = LSTM(128, activation='relu')(layer)
    layer = Dense(512, activation='relu')(layer)
    layer = Dense(512, activation='relu')(layer)
    layer = Dense(128, activation='relu')(layer)
    
    output_layer = Dense(10, activation='softmax')(layer)
    
    classifier = models.Model(input_layer, output_layer)
    
    classifier.compile(optimizer=optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return classifier

In [69]:
classifier = create_lstm_model()
train_model(classifier=classifier, X_data=X_data_tfidf_svd, y_data=y_data_n, X_test=X_test_tfidf_svd, y_test=y_test_n, is_neuralnet=True)

Train on 30383 samples, validate on 3376 samples
Epoch 1/10
30383/30383 [==============================] - 4s 144us/step - loss: 2.1759 - acc: 0.1704 - val_loss: 1.9528 - val_acc: 0.2346
Epoch 2/10
30383/30383 [==============================] - 3s 115us/step - loss: 1.6678 - acc: 0.3642 - val_loss: 1.4039 - val_acc: 0.4591
Epoch 3/10
30383/30383 [==============================] - 3s 112us/step - loss: 1.3295 - acc: 0.4862 - val_loss: 1.2670 - val_acc: 0.5092
Epoch 4/10
30383/30383 [==============================] - 4s 118us/step - loss: 1.2106 - acc: 0.5463 - val_loss: 1.2173 - val_acc: 0.5367
Epoch 5/10
30383/30383 [==============================] - 3s 113us/step - loss: 1.1339 - acc: 0.5894 - val_loss: 1.1116 - val_acc: 0.6022
Epoch 6/10
30383/30383 [==============================] - 3s 113us/step - loss: 1.0748 - acc: 0.6167 - val_loss: 1.0639 - val_acc: 0.6336
Epoch 7/10
30383/30383 [==============================] - 4s 117us/step - loss: 1.0265 - acc: 0.6442 - val_loss: 1.0948 - v

### GRU 

In [70]:
def create_gru_model():
    input_layer = Input(shape=(300,))
    
    layer = Reshape((10, 30))(input_layer)
    layer = GRU(128, activation='relu')(layer)
    layer = Dense(512, activation='relu')(layer)
    layer = Dense(512, activation='relu')(layer)
    layer = Dense(128, activation='relu')(layer)
    
    output_layer = Dense(10, activation='softmax')(layer)
    
    classifier = models.Model(input_layer, output_layer)
    
    classifier.compile(optimizer=optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return classifier

In [81]:
classifier = create_gru_model()
train_model(classifier=classifier, X_data=X_data_tfidf_svd, y_data=y_data_n, X_test=X_test_tfidf_svd, y_test=y_test_n, is_neuralnet=True, n_epochs=10)

Train on 30383 samples, validate on 3376 samples
Epoch 1/10
30383/30383 [==============================] - 4s 144us/step - loss: 2.2465 - acc: 0.1557 - val_loss: 2.1616 - val_acc: 0.1505
Epoch 2/10
30383/30383 [==============================] - 3s 100us/step - loss: 1.7984 - acc: 0.3081 - val_loss: 1.4678 - val_acc: 0.4188
Epoch 3/10
30383/30383 [==============================] - 3s 101us/step - loss: 1.3495 - acc: 0.4750 - val_loss: 1.2717 - val_acc: 0.5193
Epoch 4/10
30383/30383 [==============================] - 3s 109us/step - loss: 1.2431 - acc: 0.5203 - val_loss: 1.2419 - val_acc: 0.5320
Epoch 5/10
30383/30383 [==============================] - 3s 95us/step - loss: 1.1760 - acc: 0.5509 - val_loss: 1.1627 - val_acc: 0.5708
Epoch 6/10
30383/30383 [==============================] - 3s 104us/step - loss: 1.1315 - acc: 0.5791 - val_loss: 1.1317 - val_acc: 0.5859
Epoch 7/10
30383/30383 [==============================] - 3s 105us/step - loss: 1.0915 - acc: 0.6021 - val_loss: 1.1042 - va

### Bidirectional RNN 

In [76]:
def create_brnn_model():
    input_layer = Input(shape=(300,))
    
    layer = Reshape((10, 30))(input_layer)
    layer = Bidirectional(GRU(128, activation='relu'))(layer)
    layer = Dense(512, activation='relu')(layer)
    layer = Dense(512, activation='relu')(layer)
    layer = Dense(128, activation='relu')(layer)
    
    output_layer = Dense(10, activation='softmax')(layer)
    
    classifier = models.Model(input_layer, output_layer)
    
    classifier.compile(optimizer=optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return classifier

In [80]:
classifier = create_brnn_model()
train_model(classifier=classifier, X_data=X_data_tfidf_svd, y_data=y_data_n, X_test=X_test_tfidf_svd, y_test=y_test_n, is_neuralnet=True, n_epochs=20)

Train on 30383 samples, validate on 3376 samples
Epoch 1/20
30383/30383 [==============================] - 7s 222us/step - loss: 1.4777 - acc: 0.5496 - val_loss: 0.7177 - val_acc: 0.7820
Epoch 2/20
30383/30383 [==============================] - 5s 163us/step - loss: 0.5405 - acc: 0.8272 - val_loss: 0.5058 - val_acc: 0.8418
Epoch 3/20
30383/30383 [==============================] - 5s 167us/step - loss: 0.4284 - acc: 0.8578 - val_loss: 0.4277 - val_acc: 0.8623
Epoch 4/20
30383/30383 [==============================] - 5s 176us/step - loss: 0.3807 - acc: 0.8745 - val_loss: 0.3902 - val_acc: 0.8732
Epoch 5/20
30383/30383 [==============================] - 5s 175us/step - loss: 0.3628 - acc: 0.8787 - val_loss: 0.3968 - val_acc: 0.8723
Epoch 6/20
30383/30383 [==============================] - 5s 161us/step - loss: 0.3540 - acc: 0.8812 - val_loss: 0.3781 - val_acc: 0.8786
Epoch 7/20
30383/30383 [==============================] - 6s 188us/step - loss: 0.3361 - acc: 0.8874 - val_loss: 0.3676 - v

## Recurrent Convolutional Neural Network 

In [13]:
# def create_rcnn_model():
#     input_layer = Input(shape=(300,))
    
#     layer = Reshape((10, 30))(input_layer)
#     layer = Bidirectional(GRU(128, activation='relu', return_sequences=True))(layer)
# #     layer = Reshape((16, 16))(layer)
# #     layer = Convolution1D(100, 3, activation="relu")(layer)
#     layer = Dense(512, activation='relu')(layer)
#     layer = Dense(512, activation='relu')(layer)
#     layer = Dense(128, activation='relu')(layer)
    
#     output_layer = Dense(10, activation='softmax')(layer)
    
#     classifier = models.Model(input_layer, output_layer)
    
#     classifier.compile(optimizer=optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
#     return classifier
def create_rcnn_model():
    input_layer = Input(shape=(300,))
    
    layer = Reshape((10, 30))(input_layer)
    layer = Bidirectional(GRU(128, activation='relu', return_sequences=True))(layer)    
    layer = Convolution1D(100, 3, activation="relu")(layer)
    layer = Flatten()(layer)
    layer = Dense(512, activation='relu')(layer)
    layer = Dense(512, activation='relu')(layer)
    layer = Dense(128, activation='relu')(layer)
    
    output_layer = Dense(10, activation='softmax')(layer)
    
    classifier = models.Model(input_layer, output_layer)
    classifier.summary()
    classifier.compile(optimizer=optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return classifier

In [110]:
classifier = create_rcnn_model()
train_model(classifier=classifier, X_data=X_data_tfidf_svd, y_data=y_data_n, X_test=X_test_tfidf_svd, y_test=y_test_n, is_neuralnet=True, n_epochs=20)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_35 (InputLayer)        (None, 300)               0         
_________________________________________________________________
reshape_31 (Reshape)         (None, 10, 30)            0         
_________________________________________________________________
bidirectional_16 (Bidirectio (None, 10, 256)           122112    
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 8, 100)            76900     
_________________________________________________________________
flatten_1 (Flatten)          (None, 800)               0         
_________________________________________________________________
dense_105 (Dense)            (None, 512)               410112    
_________________________________________________________________
dense_106 (Dense)            (None, 512)               262656    
__________

## Doc2Vec Model 

In [5]:
def get_corpus(documents):
    corpus = []
    
    for i in tqdm(range(len(documents))):
        doc = documents[i]
        
        words = doc.split(' ')
        tagged_document = gensim.models.doc2vec.TaggedDocument(words, [i])
        
        corpus.append(tagged_document)
        
    return corpus

In [6]:
train_corpus = get_corpus(X_data)


100%|██████████| 33759/33759 [00:01<00:00, 22059.46it/s]


In [7]:
test_corpus = get_corpus(X_test)

100%|██████████| 50373/50373 [00:02<00:00, 17527.29it/s]


#### Build Doc2Vec model 

In [8]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=300, min_count=2, epochs=40)
model.build_vocab(train_corpus)

In [9]:
%time model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

CPU times: user 27min 55s, sys: 14.6 s, total: 28min 9s
Wall time: 15min 9s


#### Get vector 

In [10]:
X_data_vectors = []
for x in train_corpus:
    vector = model.infer_vector(x.words)
    X_data_vectors.append(vector)

In [11]:
X_test_vectors = []
for x in test_corpus:
    vector = model.infer_vector(x.words)
    X_test_vectors.append(vector)

In [ ]:
classifier = create_dnn_model()
train_model(classifier=classifier, X_data=np.array(X_data_vectors), y_data=y_data_n, X_test=(X_test_vectors), y_test=y_test_n, is_neuralnet=True, n_epochs=5)